<a href="https://colab.research.google.com/github/bessabra/bessabra/blob/main/DadosMixSei.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [9]:
import pandas as pd
from datetime import datetime, timedelta


basedf = pd.read_csv('/content/drive/MyDrive/Progressão Professores/Versões v13.12/Dados Funcionais v13.12.csv')
baserq = pd.read_csv('/content/drive/MyDrive/Progressão Professores/Versões v13.12/LISTA_SEI_PROFESSORES RR RO AP v15.12.2.csv')

In [10]:
baserq = baserq.rename(
    columns = {'INTERESSADO':'NOME'}
    )

basedf['INATIVIDADE'] = pd.to_datetime(basedf['INATIVIDADE'], errors="coerce")
basedf['OBITO'] = pd.to_datetime(basedf['OBITO'], errors="coerce")

baserq['DATA DO REQUERIMENTO'] = pd.to_datetime(baserq['DATA DO REQUERIMENTO'], errors="coerce")

In [11]:
# Junta as planilhas basedf e baserq pra identificar correspondência de CPF, SIAPE e NOME
mixResultadoGeral = basedf.merge(
    baserq, on=['CPF', 'SIAPE', 'NOME'], 
    how='right', 
    suffixes=['', '_'], 
    indicator=True
    )

# Adiciona uma coluna com legendas para cada situação
mixResultadoGeral.loc[(mixResultadoGeral['_merge'] == 'right_only'), 'CONDIÇÃO'] = 'Servidor não encontrado no sistema'
mixResultadoGeral.loc[(mixResultadoGeral['_merge'] == 'left_only'), 'CONDIÇÃO'] = 'Servidor não fez requerimento'
mixResultadoGeral.loc[(mixResultadoGeral['_merge'] == 'both'), 'CONDIÇÃO'] = 'Servidor encontrado no sistema'

# Cria uma nova planilha somente com os servidores não encontrados na CACOPOSPRO v13.12
servNao = mixResultadoGeral.loc[(mixResultadoGeral['_merge'] == 'right_only')]
servNao.to_excel('servNaoResultadoGeralv19.12.xlsx')

In [12]:
# Cria uma nova planilha somente com os servidores com requerimento e que foram encontrados na CACOPOSPRO v13.12
servEncontrados = basedf.merge(
    baserq, on=['CPF', 'SIAPE', 'NOME'], 
    how='inner', 
    suffixes=['', '_'], 
    indicator=True
    )

servEncontrados.loc[(servEncontrados['_merge'] == 'both'), 'CONDIÇÃO'] = 'Servidor encontrado no sistema'

# servEncontrados.drop('_merge', axis = 1, inplace=True)
# servEncontrados.drop('NIVEL', axis = 1, inplace=True)
# servEncontrados.drop('CLASSE', axis = 1, inplace=True)

servEncontrados.to_excel('servEncontradosv19.12.xlsx')